<a href="https://colab.research.google.com/github/ilham-mukti/Machine-Learning/blob/main/data_teks/Document_Similarity_Data_Tirto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import nltk
nltk.download("stopwords")
nltk.download("punkt")

from sklearn.metrics.pairwise import cosine_distances, cosine_similarity
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from string import punctuation
from sklearn.feature_extraction.text import TfidfVectorizer

sw_indo = stopwords.words("indonesian") + list(punctuation)

In [20]:
datas = pd.read_csv("/content/data_tirto.csv")
datas.head()

,id_berita,judul_berita,isi_berita,nama_penulis,date_created,url_artikel
0,1259889,Jokowi Sindir ASN yang Minta Dilayani: Seperti...,Presiden Jokowi menilai para aparatur sipil ne...,Andrian Pratama Taher,27/07/2021 10:55,https://tirto.id/jokowi-sindir-asn-yang-minta-...
1,1259829,Larangan Demo Jokowi End Game & Bahaya Doxing ...,"Jumat, 23 Juli 2021, sekira pukul 23-an, ada t...",Adi Briantika,26/07/2021 17:24,https://tirto.id/larangan-demo-jokowi-end-game...
2,1259921,"Jokowi: Mahasiswa Punya Talenta, Bisa Berprofe...",Presiden Joko Widodo meminta agar perguruan ti...,Andrian Pratama Taher,27/07/2021 13:36,https://tirto.id/jokowi-mahasiswa-punya-talent...
3,1259902,Tjahjo Minta Seluruh ASN Ikuti Arahan Jokowi-M...,Menteri Pendayagunaan Aparatur Negara dan Refo...,Andrian Pratama Taher,27/07/2021 12:30,https://tirto.id/tjahjo-minta-seluruh-asn-ikut...
4,1259792,"Jokowi, Menkes dan Mendag Disomasi Imbas Kelan...",Koalisi masyarakat sipil menyomasi Presiden Jo...,Irwan Syambudi,26/07/2021 13:41,https://tirto.id/jokowi-menkes-dan-mendag-diso...


In [21]:
datas.shape

(1303, 6)

In [22]:
datas.drop_duplicates(subset='id_berita', inplace=True, ignore_index=True)

In [23]:
datas.shape

(863, 6)

# Ekstrak TFIDF

In [24]:
tfidf = TfidfVectorizer(ngram_range=(1,2), tokenizer=word_tokenize, stop_words=sw_indo)
tfidf_matrix = tfidf.fit_transform(datas.isi_berita)
tfidf_matrix

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['``'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


<863x243099 sparse matrix of type '<class 'numpy.float64'>'
	with 475069 stored elements in Compressed Sparse Row format>

# Document Similarity

In [27]:
def document_similarity(index, matrix, topk=10):
  sim = cosine_similarity(matrix[index], matrix)
  sorted_sim = sim[0].argsort()
  berita_yang_sama = [datas.isi_berita[index][:200] for index in reversed(sorted_sim[-topk:])]
  return(berita_yang_sama)

In [30]:
data_ke = 3
document_similarity(data_ke, tfidf_matrix)

['Menteri Pendayagunaan Aparatur Negara dan Reformasi Birokrasi (PAN-RB) Tjahjo Kumolo meminta kepada seluruh aparatur sipil negara (ASN) untuk bersatu padu dalam penanganan COVID-19. Ia meminta kepada ',
 'Presiden Jokowi menilai para aparatur sipil negara (ASN) di masa depan harus mampu melayani masyarakat. Ia tidak mau ada ASN justru bermental seperti aparatur di masa kolonial yang ingin dilayani oleh',
 'Selama penerapan aturan\xa0PPKM\xa0Level 4, Kementerian Pendayagunaan Aparatur Negara dan Reformasi Birokrasi (PANRB) menerbitkan aturan baru terkait penyesuaian sistem kerja aparatur sipil negara (ASN) dan',
 'Badan Kepegawaian Negara (BKN) menginformasikan bahwa pendaftaran CASN 2021 termasuk CPNS 2021 dan PPPK 2021 di www.sscasn.bkn.go.id diperpanjang hingga 26 Juli 2021\xa0pukul 23:59 WIB.Informasi tersebut',
 'Pemerintah Kabupaten Boyolali di Provinsi Jawa Tengah mulai Senin (14/9/2020) akan menerapkan kebijakan kerja dari rumah atau WFH bagi aparatur sipil negara (ASN) dalam 

In [31]:
def extract_keyword_tfidf(doc, tfidf, topk=10):
  matrix = tfidf.transform([doc])
  vocab = tfidf.get_feature_names()

  sorted_tfidf = matrix[0].toarray()[0].argsort()
  return([vocab[idx] for idx in reversed(sorted_tfidf[-topk:])])

In [35]:
text = datas.isi_berita[2]
extract_keyword_tfidf(text, tfidf)

['mahasiswa',
 'kampus',
 'pengetahuan',
 'pengetahuan keterampilan',
 'keterampilan',
 'industri',
 'jokowi lembaga',
 'kampus merdeka',
 'menyongsong',
 'praktisi']